# Semianr 10 - Applied Quantitative Logistics

### Firefly Algorithm (FA)

In [ ]:
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Sphere Problem

$$
\min{z} = f_{sph}(x) = \begin{equation*}
 \sum_{i=1}^n {x_i}^2 \end{equation*}
$$

$$
x_{min} \le x_i \le x_{max}
$$

Optimal Solutions:

$$
\forall i \;
\left\{
    \begin{array}\\
        x_i^* = 0 \\
        z^* = 0 \\
    \end{array}
\right.
$$

In [ ]:
def sphere(x):
    
    global NFE
    
    if pd.isna(NFE):
        NFE = 0
        
    NFE += 1
    
    z = [item**2 for item in x]
    
    return sum(z)

In [ ]:
# Sort the population and cost (based on the cost)
def pop_sort(p, c):
    li = []
    for i in range(len(c)):
        li.append([c[i],i])
        
    li.sort()
    sort_index = []
    
    for x in li:
        sort_index.append(x[1])
    
    positions, cost = [], []
    for i in sort_index:
        positions.append(p[i])
        cost.append(c[i])
        
    return positions, cost

### FA Optimization

In [ ]:
## Problem Parameters

nVar = 5      # Number of Decision Variables

varMin = -10
varMax = 10

global NFE
NFE = 0

nfe = []

## FA Parameters
maxIteration = 500         # Maximum number of iteration

nPop = 50         # Number of Fireflies (Swarm size)

gamma = 1         # Light Absorption Coefficient

beta0 = 2         # Attraction Coefficient Base value

alpha = 0.2       # Mutation Vector Coefficient

delta = 0.05*(varMax-varMin)      # Uniform Mutation Range

m = 2

# Initialization
pop, costs = [], []

# Initialize Best Solution Ever Found
BestSol = []
BestSol_Cost = np.inf
BestCosts_list = []

for i in range(nPop):
    pop.append(list(np.random.uniform(varMin, varMax, size=nVar)))
    costs.append(sphere(pop[i]))
    
    if costs[i] < BestSol_Cost:
        BestSol = pop[i]
        BestSol_Cost = costs[i]
        
## FA Main Loop
for it in range(maxIteration):
    
    newPop = pop.copy()
    newCosts = costs.copy()
    
    for i in range(nPop):
        for j in range(nPop):
            if i == j:
                continue
            
            if costs[j] <= costs[i]:
                rij = np.linalg.norm(np.array(pop[i]) - np.array(pop[j]))
                beta = beta0 * np.exp(-gamma*(rij**m))
                e = np.random.uniform(-delta, +delta, size=nVar)
                newPop[i] = np.array(pop[i]) + beta*(np.array(pop[j])-np.array(pop[i]))+alpha*e
            
            newPop[i] = [max(item, varMin) for item in newPop[i]] 
            newPop[i] = [min(item, varMax) for item in newPop[i]]
            
            newCosts[i] = sphere(newPop[i])
            
            if newCosts[i] <= BestSol_Cost:
                BestSol = newPop[i]
                BestSol_Cost = newCosts[i]
                
    # Create merge population
    pop = pop + newPop + BestSol
    costs = costs + newCosts + BestSol_Cost
    
    # Sort the population and costs
    pop, costs = pop_sort(pop, costs)
    
    # Truncation
    pop = pop[:nPop]
    costs = costs[:nPop]
    
    BestCosts_list.append(BestSol_Cost)
    
    # store nfe
    nfe.append(NFE)
    
    print(f'Iteration {it} : NFE = {nfe[it]},  Best Cost = {BestCosts_list[it]}')

In [ ]:
plt.plot(BestCosts_list, linewidth = 3)
plt.xlabel('NFE')
plt.ylabel('Best Cost')